In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from scipy import io
import scipy.optimize as opt
from sklearn.model_selection import train_test_split
pd.options.display.float_format = '{:.2f}'.format

%matplotlib inline

## Neural networks

### Ex. 1 - Feed forward and cost function

In [4]:
# read in dataset from the course
df = scipy.io.loadmat('/Users/dorotamierzwa/Data Science/Machine Learning - Coursera/machine-learning-ex4/ex4/ex4weights.mat')

In [5]:
df['Theta1'].shape

(25, 401)

In [6]:
df['Theta2'].shape

(10, 26)

In [7]:
theta1 = df['Theta1']
theta2 = df['Theta2']

In [8]:
# read in dataset from the course
df2 = scipy.io.loadmat('/Users/dorotamierzwa/Data Science/Machine Learning - Coursera/machine-learning-ex4/ex4/ex4data1.mat')

In [9]:
X = df2['X']
y = df2['y']

In [10]:
input_layer_size  = 400
hidden_layer_size = 25
num_labels = 10

In [11]:
nn_params = np.concatenate((theta1.flatten(), theta2.flatten()))

In [12]:
nn_params.shape

(10285,)

In [13]:
nn_params = nn_params[:,np.newaxis]
nn_params.shape

(10285, 1)

In [14]:
def sigmoid(z):
    g = np.zeros(len(np.array([z])))
    g = 1 / (1 + math.e**(-z))
    return g

In [37]:
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lmbda):
    m = X.shape[0]
    a1 = np.hstack((np.ones((X.shape[0], 1)) , X))
    a2 = np.hstack((np.ones((a1.shape[0], 1)), sigmoid(np.dot(a1, theta1.T))))
    h3 = sigmoid(np.dot(a2, theta2.T))
    y_k = np.zeros((m,num_labels))
    for i in range(m):
        y_k[i, y[i]-1] = 1
    J = (1.0/m) * np.sum(np.sum((-y_k * np.log(h3)) - ((1-y_k) * np.log(1-h3))))
    J += (lmbda / (2*m)) * np.sum(np.sum(theta1[:, 1:] ** 2))
    J += (lmbda / (2*m)) * np.sum(np.sum(theta2[:, 1:] ** 2))
    
    return J

In [38]:
J = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, 0)
J

0.2876291651613189

In [39]:
# with regularization
J = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, 1)
J

0.38376985909092365

### Ex 2. Sigmoid gradient

In [18]:
def sigmoidGradient(z):
    g = sigmoid(z) * (1 - sigmoid(z))
    return g

In [19]:
z = np.array((-1, -0.5, 0, 0.5, 1))

In [20]:
sigmoidGradient(z)

array([0.19661193, 0.23500371, 0.25      , 0.23500371, 0.19661193])

### Ex 3. Randomly initialize weights

In [24]:
def randInitializeWeights(l_in, l_out):
    epsilon_init = 0.12
    weights = np.random.rand(l_out, l_in + 1) * 2 * epsilon_init - epsilon_init
    return weights

In [25]:
initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

In [26]:
initial_Theta1.shape

(25, 401)

In [27]:
initial_nn_params = np.concatenate((initial_Theta1.flatten(), initial_Theta2.flatten()))

In [28]:
initial_nn_params.shape

(10285,)

### Ex 4. Backpropagation

In [53]:
def nnBacpropagation(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lmbda):
    theta1 = nn_params[:((input_layer_size+1) * hidden_layer_size)].reshape(hidden_layer_size, input_layer_size+1)
    theta2 = nn_params[((input_layer_size+1) * hidden_layer_size ):].reshape(num_labels, hidden_layer_size+1)
    
    m = X.shape[0]
    J = 0
    X = np.hstack((np.ones((m,1)),X))
    y_k = np.zeros((m,num_labels))
    
    a1 = sigmoid(np.dot(X, theta1.T))
    a1 = np.hstack((np.ones((m,1)), a1)) 
    a2 = sigmoid(np.dot(a1, theta2.T)) 
    
    for i in range(m):
        y_k[i, y[i]-1] = 1
    for j in range(num_labels):
        J = J + sum(-y_k[:,j] * np.log(a2[:,j]) - (1-y_k[:,j])*np.log(1-a2[:,j]))
    
    cost = 1/m* J
    reg_J = cost + lmbda/(2*m) * (np.sum(theta1[:,1:]**2) + np.sum(theta2[:,1:]**2))
        
    grad1 = np.zeros((theta1.shape))
    grad2 = np.zeros((theta2.shape))
    
    for i in range(m):
        xi = X[i,:] 
        a1i = a1[i,:] 
        a2i = a2[i,:] 
        d2 = a2i - y_k[i,:]
        d1 = np.dot(theta2.T, d2.T) * sigmoidGradient(np.hstack((1, np.dot(xi, theta1.T))))
        grad1 = grad1 + np.dot(d1[1:][:,np.newaxis], xi[:,np.newaxis].T)
        grad2 = grad2 + np.dot(d2.T[:,np.newaxis], a1i[:,np.newaxis].T)
        
    grad1 = 1/m * grad1
    grad2 = 1/m * grad2
    
    grad1_reg = grad1 + (lmbda/m) * np.hstack((np.zeros((theta1.shape[0],1)),theta1[:,1:]))
    grad2_reg = grad2 + (lmbda/m) * np.hstack((np.zeros((theta2.shape[0],1)),theta2[:,1:]))
    
    return grad1, grad2, reg_J, grad1_reg, grad2_reg

In [56]:
def gradientDescentnn(X, y, initial_nn_params, alpha, num_iters, lmbda, input_layer_size, hidden_layer_size, 
                      num_labels):
    theta1 = initial_nn_params[:((input_layer_size+1) * hidden_layer_size)].reshape(hidden_layer_size, input_layer_size+1)
    theta2 = initial_nn_params[((input_layer_size+1) * hidden_layer_size ):].reshape(num_labels, hidden_layer_size+1)
    
    m = len(y)
    J_history =[]
    
    for i in range(num_iters):
        nn_params = np.append(theta1.flatten(),theta2.flatten())
        grad1, grad2, cost = nnBacpropagation(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lmbda)[:3]
        theta1 = theta1 - (alpha * grad1)
        theta2 = theta2 - (alpha * grad2)
        J_history.append(cost)
    
    nn_paramsFinal = np.append(theta1.flatten(),theta2.flatten())
    
    return nn_paramsFinal, J_history

In [57]:
nnTheta, nnJ_history = gradientDescentnn(X, y, initial_nn_params, 0.8, 800, 1, input_layer_size, 
                                         hidden_layer_size, num_labels)
theta1 = nnTheta[:((input_layer_size+1) * hidden_layer_size)].reshape(hidden_layer_size, input_layer_size+1)
theta2 = nnTheta[((input_layer_size+1) * hidden_layer_size ):].reshape(num_labels, hidden_layer_size+1)

In [58]:
def predict(theta1, theta2, X):
    m= X.shape[0]
    X = np.hstack((np.ones((m,1)),X))
    
    a1 = sigmoid(np.dot(X, theta1.T))
    a1 = np.hstack((np.ones((m,1)), a1)) # hidden layer
    a2 = sigmoid(np.dot(a1, theta2.T) # output layer
    return np.argmax(a2, axis=1)+1

In [60]:
pred = predict(theta1, theta2, X)
print("Training set accuracy:", sum(pred[:, np.newaxis]==y)[0]/5000*100, "%")

Training set accuracy: 94.6 %
